In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from PIL import Image
from google.cloud import vision
from google.cloud.vision import types
import time
import numpy as np
import pytesseract as ocr
import cv2

In [ ]:
driver = webdriver.Chrome('/home/lucas.cardoso/fastfinger-tensorflow/chromedriver_linux64/chromedriver')
driver.get('https://10fastfingers.com/login')

In [ ]:
login = input("Email: ")
password = input ("Password: ")

In [ ]:
driver.find_element('id', 'UserEmail').send_keys(login)
driver.find_element('id', 'UserPassword').send_keys(password)
driver.find_element('id', 'login-form-submit').click()

In [ ]:
while True:
    try:
        highlightWord = driver.find_elements_by_class_name("highlight")[0]   
    except:
        break
    driver.find_element('id','inputfield').send_keys(highlightWord.text + " ")
    time.sleep(0.075)

In [ ]:
driver.get("https://10fastfingers.com/anticheat/view/1/1")
driver.find_element('id', 'start-btn').click()
driver.set_window_size(1050, 1000)
print (driver.get_window_size())

In [ ]:
driver.save_screenshot("screenshot.png")

In [ ]:
from PIL import Image
imageObject  = Image.open("screenshot.png")
cropped = imageObject.crop((300,190,880,320))
cropped.save("screenshot.png")

In [ ]:
imagem = Image.open('screenshot.png').convert('RGB')
npimagem = np.asarray(imagem).astype(np.uint8)  
npimagem[:, :, 0] = 0
npimagem[:, :, 2] = 0
im = cv2.cvtColor(npimagem, cv2.COLOR_RGB2GRAY) 

ret, thresh = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 

binimagem = Image.fromarray(thresh) 

phrase = ocr.image_to_string(binimagem, lang='por')
print(phrase) 


In [ ]:
import os
import io

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/lucas.cardoso/Downloads/fastfingers-67e42e3ddf3d.json"
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.join(
    os.path.dirname('screenshot.png'))

# Loads the image into memory
with io.open('screenshot.png', 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)

In [ ]:
textarea = driver.find_element('id', 'word-input')
textarea.send_keys (phrase)
driver.find_element('id','submit-anticheat').click()